In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt

from skimage import io
from skimage.measure import label

from shutil import copy
import numpy as np

In [ ]:
# get the images
base_path = Path("../../midap/training/midap_training/")

fluor_train_images_raw = sorted(base_path.glob("**/[!PH]*/*/*raw.tif"))
fluor_train_images_seg = sorted(base_path.glob("**/[!PH]*/*/*seg.tif"))
ph_train_images_raw = sorted(base_path.glob("**/PH/*/*raw.tif"))
ph_train_images_seg = sorted(base_path.glob("**/PH/*/*seg.tif"))

In [ ]:
def scale_pixel_vals(img: np.ndarray):
    """
    Scales the values of the pixels of an image such that they are between 0 and 1
    :param img: The input image as array
    :returns: The images with pixels scales between 0 and 1
    """

    img = np.array(img)
    return ((img - img.min()) / (img.max() - img.min()))

In [ ]:
# define the new base and save all images there
new_base = Path("./fine_tune_data/")

for name, raw_imgs, seg_imgs in [("fluor", fluor_train_images_raw, fluor_train_images_seg),
                                 ("PH", ph_train_images_raw, ph_train_images_seg)]:
    print(f"Starting with {name}")
    current_dir = new_base.joinpath(name)
    current_dir.mkdir(exist_ok=True, parents=True)
    for raw, seg in zip(raw_imgs, seg_imgs):
        # raw images
        raw_name = raw.name.replace("_raw", "")
        new_raw = current_dir.joinpath(raw_name)
        raw_img = scale_pixel_vals(io.imread(raw))
        io.imsave(new_raw, raw_img)

        # segmentations
        seg_im = io.imread(seg)
        label_im = label(seg_im)

        # plot the segs for sanity check
        plt.figure()
        plt.title(f"Number of cells: {label_im.max()}")
        plt.imshow(label_im)
        plt.show()

        new_seg = current_dir.joinpath(seg.name)
        io.imsave(new_seg, label_im.astype(np.uint16))